# Training with Tensorflow Keras - CrackNet

In [1]:
%load_ext autoreload
%autoreload 2

import sys
WORKSPACE_BASE_PATH="/tf/notebooks/" # Parent directory containing src, checkpoints, models, etc.
CODE_BASE_PATH="/tf/notebooks/src/" # Path were components are stored.
DATA_BASE_PATH="/tf/notebooks/data/" # Directory with data in case it is not inside WORKSPACE BASE path.
sys.path.append(CODE_BASE_PATH)
from cnnmodels import cracknet

## Model definition

In [17]:
from tensorflow.keras.utils import plot_model

model = cracknet.build_cracknet()
model.summary()
plot_model(model, show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 110, 110, 32)      11648     
_________________________________________________________________
batch_normalization_49 (Batc (None, 110, 110, 32)      128       
_________________________________________________________________
elu_53 (ELU)                 (None, 110, 110, 32)      0         
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 52, 52, 32)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 42, 42, 48)        185904    
_________________________________________________________________
batch_normalization_50 (Batc (None, 42, 42, 48)        192       
_________________________________________________________________
elu_54 (ELU)                 (None, 42, 42, 48)        0         
__________

## Training

In [18]:
MODEL_NAME="cracknet_cracks8020"

In [ ]:
cracknet.train_cracknet(  model,
            target_size=(120,120),
            dataset_path=DATA_BASE_PATH+"/datasets/cracks_splitted8020/",
            training_path_prefix="train_set",
            test_path_prefix="test_set",
            history_file_path=WORKSPACE_BASE_PATH+"/training_logs/",
            history_filename=MODEL_NAME+".csv",
            checkpoint_path=WORKSPACE_BASE_PATH+"/model-checkpoints/",
            checkpoint_prefix=MODEL_NAME,
            number_of_epochs=30,
            tensorboard_log_path=WORKSPACE_BASE_PATH+"/tensorboard_logs/",
)

Found 32000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Epoch 1/30
 362/1000 [=========>....................] - ETA: 3:11 - loss: 0.2625 - acc: 0.9578

## Training Report

In [ ]:
import matplotlib
from utils.plotutils import plot_learning_curves_from_history_file
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10, 6)
fig = plot_learning_curves_from_history_file(WORKSPACE_BASE_PATH+"/training_logs/"+MODEL_NAME+".csv")

## Export model to SavedModelFormat

In [ ]:
import glob
import os

list_of_files = glob.glob(WORKSPACE_BASE_PATH+'/model-checkpoints/*.hdf5') 
CHECKPOINT_FILE = max(list_of_files, key=os.path.getctime) # last checkpoint
VERSION=1
print(CHECKPOINT_FILE)

In [ ]:
from utils.export_tools import convert_from_keras_to_savedmodel

convert_from_keras_to_savedmodel(
    input_filename=CHECKPOINT_FILE,
    export_path=WORKSPACE_BASE_PATH+'/models/'+MODEL_NAME+"/"+str(VERSION)
)